**LeNet-5**

In [ ]:
!ls /kaggle/input/Kannada-MNIST

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# %tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras
print(keras.__version__)

In [ ]:
train_set = pd.read_csv("../input/Kannada-MNIST/train.csv")
# print(type(train_set))
train_array = train_set.values
y_train, x_train = train_array[:, :1], train_array[:, 1:]
# print(y_train, y_train.shape,"\n", x_train, x_train.shape)
# print(x_train.dtype, x_train.min(), x_train.max())

In [ ]:
test_set = pd.read_csv("../input/Kannada-MNIST/test.csv")
# print(type(test_set))
test_array = test_set.values
y_test, x_test = test_array[:, :1], test_array[:, 1:]

Нармалізацыя

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler((-1, 1), False)
scaler = MinMaxScaler((0, 1), False)
print(scaler.fit(x_train))
# print(scaler.data_max_)
x_train = scaler.transform(x_train)
# print(scaler.data_max_)
# print(x_train)
print(x_train.min(), x_train.max())

І для тэставых

In [ ]:
print(x_test.min(), x_test.max())
scaler = MinMaxScaler((0, 1), False)
scaler.fit(x_test)
x_test = scaler.transform(x_test)
print(x_test.min(), x_test.max())

Б'ем на трэйн і валідацыю

In [ ]:
border = 50000
x_train, x_valid, y_train, y_valid = x_train[:border], x_train[border:], y_train[:border], y_train[border:]

Пераўтвараем у патрэбны 4Д-фармат

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 10)
y_valid = np_utils.to_categorical(y_valid, 10)

print(x_train.shape)
print(x_valid.shape)
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
# print(x_train)
x_valid = x_valid.reshape(x_valid.shape[0], 28,28,1)

x_test = x_test.reshape(x_test.shape[0], 28,28,1)
print(x_train.shape)
print(x_valid.shape)

Будуем архітэктуру

In [ ]:
from keras.models import Sequential
from keras import models, layers
# import keras

from keras.callbacks import CSVLogger, ModelCheckpoint

model = Sequential()

model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28,28,1),
                        padding='same'))

model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))


model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

#Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

model.add(layers.Dense(84, activation='tanh'))

model.add(layers.Dense(10, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=['accuracy'])

model.summary()

Вучым

In [ ]:
BATCH_SIZE = 16
EPOCHS = 30

model.fit(
          x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          # validation_data=(x_valid - 0.5, y_valid),
          validation_data=(x_valid, y_valid),
          callbacks=[
              CSVLogger('log.csv'),
              ModelCheckpoint('model.h5', save_best_only=True),
          ])

In [ ]:
# model = keras.models.load_model('model.h5')

# pred_probas = model.predict(x_test - 0.5, batch_size=16)
pred_probas = model.predict(x_test, batch_size=16)

prediction = pred_probas.argmax(axis=1)


In [ ]:
submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = prediction

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!ls